In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
# !pip install snowflake-connector-python
!pip uninstall cloudpickle -y
!pip install cloudpickle==2.2.1

In [15]:
from snowflake.snowpark import Session

In [16]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWINSIGHT"

In [17]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=REGISTRY_DATABASE_NAME,
                         schema=REGISTRY_SCHEMA_NAME, 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [18]:
session = get_session()
# session.close()
session

In [19]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [ ]:
!pip list | grep -i snow

NameError: name 'Registry' is not defined

In [ ]:
# model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [21]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [22]:
df = registry.show_models()
df
# type(df)

""


# Get model list from snowpark registry

In [23]:
model_list = registry.models()
model_list

[]

# Get model details from registry

In [ ]:
m =registry.get_model("my_model")
m

# Get model versions

In [ ]:
df_version = m.show_versions()
df_version

# Set default model version

In [ ]:
# default_version = m.default
m.default = "V1"

# Get default model version details from model-registry

In [ ]:
mv = m.default
mv

In [ ]:
x

In [ ]:
#Load default vesion as python object
# clf = mv.load()

# Run prediction on scikit-learn model

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = mv.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


In [ ]:
type(x_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

# Deploy model with custom class

In [ ]:
from snowflake.ml.model import custom_model

In [ ]:
class ExamplePipelineModel(custom_model.CustomModel):
#     def __init__(self, context: ModelContext) -> None:
#         super().__init__(context)
#         v = int(open(context.path('config')).read())
#         self.bias = json.loads(v)['bias']

    @custom_model.inference_api
    def run(self, data_list: list) -> pd.DataFrame:
#         features = self.context.model_ref('feature_preproc').transform(input)
#         model_output = self.context.model_ref('m2').predict(
#             self.context.model_ref('m1').predict(features)
#         )
#         return pd.DataFrame({
#             'output': model_output + self.bias})
        data_array = np.asarray(data_list)
        try:
            prediction = model.predict(data_array)
        except:
            prediction = model.predict(data_array.reshape(1, -1))
        return prediction.tolist()

In [ ]:
!pip freeze | grep cloud

# Train & register model

In [24]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [25]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

,short_name,nationality,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Argentina,94,94,565000,95500000,32,170,72,88,...,75,96,33,37,26,6,11,15,14,8
1,Cristiano Ronaldo,Portugal,93,93,405000,58500000,34,187,83,84,...,85,95,28,32,24,7,11,15,14,11
2,Neymar Jr,Brazil,92,92,290000,105500000,27,175,68,87,...,90,94,27,26,29,9,9,15,15,11
3,J. Oblak,Slovenia,91,93,125000,77500000,26,188,87,13,...,11,68,27,12,18,87,92,78,90,89
4,E. Hazard,Belgium,91,91,470000,90000000,28,175,74,81,...,88,91,34,27,22,11,12,6,8,8
5,K. De Bruyne,Belgium,91,91,370000,90000000,28,181,70,93,...,79,91,68,58,51,15,13,5,10,13
6,M. ter Stegen,Germany,90,93,250000,67500000,27,187,85,18,...,25,70,25,13,10,88,85,88,88,90
7,V. van Dijk,Netherlands,90,91,200000,78000000,27,193,92,53,...,62,89,91,92,85,13,10,13,11,11
8,L. Modric,Croatia,90,90,340000,45000000,33,172,66,86,...,82,92,68,76,71,13,9,7,14,9
9,M. Salah,Egypt,90,90,240000,80500000,27,175,71,79,...,77,91,38,43,41,14,14,9,11,14


In [29]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [27]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

GradientBoostingRegressor()

In [ ]:
type(gbm_default)

In [30]:
mv = registry.log_model(gbm_default,
                   model_name="sample_sklearn_model",
                   version_name="v1",
                   conda_dependencies=["scikit-learn", "cloudpickle"],
                   comment="My awesome ML model",
                    python_version="3.9",
                   metrics={"score": 96},
                   sample_input_data=X_train
                       )

ValueError: (2110) Signatures and sample_input_data both cannot be None at the same time for this kind of model.

In [ ]:
def pickle_dumps(obj, file_path):
    import cloudpickle
    out = open(file_path, "wb")
    cloudpickle.dump(obj, out)
    out.close()

In [ ]:
pickle_dumps(gbm_default, "/notebooks/notebooks/ml_model_1")

# Create stage in snowflake-model-registry

In [ ]:
# query = """
#         create or replace stage sample_model_serving_udf_stg
#         directory = (enable = true)
#         copy_options = (on_error='skip_file')
#         """

# session.sql(query)



In [6]:
# To see the stages
session.sql("show stages").show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"  |"name"  |"database_name"  |"schema_name"  |"url"  |"has_credentials"  |"has_encryption_key"  |"owner"  |"comment"  |"region"  |"type"  |"cloud"  |"notification_channel"  |"storage_integration"  |"endpoint"  |"owner_role_type"  |"directory_enabled"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|              |        |                 |               |       |                   |                      |         |           |          |        |         |                        |            

# Upload model file on ML_MODELS stage 

In [14]:
#create the stage for storing the ML models
# session.sql("USE MODEL_REGISTRY_SNOWSIGHT;")
session.sql('CREATE OR REPLACE STAGE ML_MODELS')

In [ ]:
session.file.put(
    "/notebooks/notebooks/ml_model_2", "@ML_MODELS", auto_compress=False, overwrite=True
)

# Read model file from stage

In [ ]:
session.clear_imports()
session.clear_packages()

#Register above uploded model as import of UDF
session.add_import("@ML_MODELS/ml_model_2")

#map packege dependancies
session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")

In [ ]:
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
import snowflake.snowpark.functions as F
from typing import  Optional



def read_file(filename):
    import cloudpickle
    import sys
    import os
    
    #where all imports located at
    import_dir = sys._xoptions.get("snowflake_import_directory")

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = cloudpickle.load(file)
            return m

#register UDF
@F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@ML_MODELS')
def read_model_from_file(file_name:Optional[str]) -> str:
    import numpy as np
    
    # later we will input train data as JSON object
    # hance, we have to convert JSON object as pandas DF
    data_array = np.asarray(data_list)
    pipeline = read_file(file_name)
    print("type of pickle object =", type(pipeline))
#     dict_mod["model_obj"] = pipeline
    return "pipeline"
#     return pipeline.predict(data_array).to_list()




# @F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@ML_MODELS')
# def read_model_from_file(file_name:Optional[str]) -> str:
#     import numpy as np
    
#     # later we will input train data as JSON object
#     # hance, we have to convert JSON object as pandas DF
#     data_array = np.asarray(data_list)
#     pipeline = read_file(file_name)
#     print("type of pickle object =", type(pipeline))
# #     dict_mod["model_obj"] = pipeline
#     return pipeline
# #     return pipeline.predict(data_array).to_list()

In [ ]:
session.sql("DESC FUNCTION READ_MODEL_FROM_FILE("ml_model")").show()

In [ ]:
model_udf = read_model_from_file("ml_model_2")
# model_udf = read_file("ml_model_2")

In [ ]:
model_udf

In [ ]:
registry.

In [ ]:
import joblib

In [ ]:
model_obj = joblib.load(read_model_from_file("ml_model_2"))

In [ ]:
import numpy as np

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = model_obj.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

# 